In [1]:
import os
import sys
import errno
import re
import glob
import csv
import os.path as op
import json
import math
from datetime import datetime, timedelta, date
from IPython.display import display, clear_output, HTML
import numpy as np
from pathlib import Path
import pandas as pd
from osgeo import osr, ogr, gdal
from osgeo.gdalnumeric import *
from osgeo.gdalconst import *
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import matplotlib.gridspec as pltg
import seaborn as sn
from matplotlib.colors import LogNorm
import calendar
import pyproj
from pyproj import Proj, transform
import warnings
import rasterio

import pickle
from rasterio.warp import calculate_default_transform , reproject, Resampling 
from rasterio.mask import mask
from rasterio.merge import merge
from shapely.geometry import mapping, shape
import statsmodels.api as sm
import statsmodels
import scipy.stats as st
from sklearn.metrics import r2_score, mean_squared_error
import geopandas as gpd
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 



In [2]:
def getDateFromStr(N):
    sepList = ["","-","_","/"]
    datev = ''
    for s in sepList :
        found = re.search('\d{4}'+ s +'\d{2}'+ s +'\d{2}', N)
        if found != None :
            datev = datetime.strptime(found.group(0), '%Y'+ s +'%m'+ s +'%d').date()
            break
    return datev

def getTimeFromStr(N,):
    sepList = ["","-","_"]
    HHMMSS = ''
    for s in sepList :
        found = re.search('-'+'\d{2}'+ s +'\d{2}'+ s +'\d{2}'+'-', N)
        if found != None :
            HHMMSS = datetime.strptime(found.group(0), '-'+'%H'+ s +'%M'+ s +'%S'+'-').time()
            break
    return HHMMSS

def reproject(inEPSG,outEPSG,x1,y1):
    
    inProj = Proj(init='EPSG:' + inEPSG)
    outProj = Proj(init='EPSG:'+ outEPSG)
    x2,y2 = transform(inProj,outProj,x1,y1)
    
    return x2, y2

def getCoords(G):
    
    
    GT = G.GetGeoTransform()
    minx = GT[0]
    maxy = GT[3]
    maxx = minx + GT[1] * G.RasterXSize
    miny = maxy + GT[5] * G.RasterYSize
    
    return minx, maxy, maxx, miny

def mkdir_p(dos):
    try:
        os.makedirs(dos)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(dos):
            pass
        else:
            raise
            
def getTileFromStr(N):

    tile = ''
    found = re.search('\d{2}' +'[A-Z]{3}', N)
    if found != None : tile = found.group(0)
    return tile



In [3]:
S2_tiles={
    "PYR":
    {
        "30TXN":{'EPSG':'32630','MINX':600000,'MINY':4690200,'MAXX':709800,'MAXY':4800000},
        '30TYN':{'EPSG':'32630','MINX':699960,'MINY':4690200,'MAXX':809760,'MAXY':4800000},
        '31TCH':{'EPSG':'32631','MINX':300000,'MINY':4690200,'MAXX':409800,'MAXY':4800000},
        '31TDH':{'EPSG':'32631','MINX':399960,'MINY':4690200,'MAXX':509760,'MAXY':4800000}
    },
    "ALP":
    {
        "31TGJ":{'EPSG':'32631','MINX':699960,'MINY':4790220,'MAXX':809760,'MAXY':4900020},
        '31TGK':{'EPSG':'32631','MINX':699960,'MINY':4890240,'MAXX':809760,'MAXY':5000040},
        '31TGL':{'EPSG':'32631','MINX':699960,'MINY':4990200,'MAXX':809760,'MAXY':5100000},
        '31TGM':{'EPSG':'32631','MINX':699960,'MINY':5090220,'MAXX':809760,'MAXY':5200020},
        "32TLP":{'EPSG':'32632','MINX':300000,'MINY':4790220,'MAXX':409800,'MAXY':4900020},
        '32TLQ':{'EPSG':'32632','MINX':300000,'MINY':4890240,'MAXX':409800,'MAXY':5000040},
        '32TLR':{'EPSG':'32632','MINX':300000,'MINY':4990200,'MAXX':409800,'MAXY':5100000},
        '32TLS':{'EPSG':'32632','MINX':300000,'MINY':5090220,'MAXX':409800,'MAXY':5200020}
    }
}


epsg_list={
    "30TXN":{'EPSG':'32630'},
    '30TYN':{'EPSG':'32630'},
    '31TCH':{'EPSG':'32631'},
    '31TDH':{'EPSG':'32631'},
    "31TGJ":{'EPSG':'32631'},
    '31TGK':{'EPSG':'32631'},
    '31TGL':{'EPSG':'32631'},
    '31TGM':{'EPSG':'32631'},
    "32TLP":{'EPSG':'32632'},
    '32TLQ':{'EPSG':'32632'},
    '32TLR':{'EPSG':'32632'},
    '32TLS':{'EPSG':'32632'}
}


S2_4326_tiles={
    "PYR":
    {
        "30TXN":{'EPSG':'32630','MINX':reproject('32630','4326',600000,4690200)[0],'MINY':reproject('32630','4326',600000,4690200)[1],'MAXX':reproject('32630','4326',709800,4800000)[0],'MAXY':reproject('32630','4326',709800,4800000)[1]
                },
        '30TYN':{'EPSG':'32630','MINX':reproject('32630','4326',699960,4690200)[0],'MINY':reproject('32630','4326',699960,4690200)[1],'MAXX':reproject('32630','4326',809760,4800000)[0],'MAXY':reproject('32630','4326',809760,4800000)[1]
                },
        '31TCH':{'EPSG':'32631','MINX':reproject('32631','4326',300000,4690200)[0],'MINY':reproject('32631','4326',300000,4690200)[1],'MAXX':reproject('32631','4326',409800,4800000)[0],'MAXY':reproject('32631','4326',409800,4800000)[1]
                },
        '31TDH':{'EPSG':'32631','MINX':reproject('32631','4326',399960,4690200)[0],'MINY':reproject('32631','4326',399960,4690200)[1],'MAXX':reproject('32631','4326',509760,4800000)[0],'MAXY':reproject('32631','4326',509760,4800000)[1]
                }
    },
    "ALP":
    {
        "31TGJ":{'EPSG':'32631','MINX':reproject('32631','4326',699960,4790220)[0],'MINY':reproject('32631','4326',699960,4790220)[1],'MAXX':reproject('32631','4326',809760,4900020)[0],'MAXY':reproject('32631','4326',809760,4900020)[1]
                },
        '31TGK':{'EPSG':'32631','MINX':reproject('32631','4326',699960,4890240)[0],'MINY':reproject('32631','4326',699960,4890240)[1],'MAXX':reproject('32631','4326',809760,5000040)[0],'MAXY':reproject('32631','4326',809760,5000040)[1]
                },
        '31TGL':{'EPSG':'32631','MINX':reproject('32631','4326',699960,4990200)[0],'MINY':reproject('32631','4326',699960,4990200)[1],'MAXX':reproject('32631','4326',809760,5100000)[0],'MAXY':reproject('32631','4326',809760,5100000)[1]
                },
        '31TGM':{'EPSG':'32631','MINX':reproject('32631','4326',699960,5090220)[0],'MINY':reproject('32631','4326',699960,5090220)[1],'MAXX':reproject('32631','4326',809760,5200020)[0],'MAXY':reproject('32631','4326',809760,5200020)[1]
                },
        "32TLP":{'EPSG':'32632','MINX':reproject('32632','4326',300000,4790220)[0],'MINY':reproject('32632','4326',300000,4790220)[1],'MAXX':reproject('32632','4326',409800,4900020)[0],'MAXY':reproject('32632','4326',409800,4900020)[1]
                },
        '32TLQ':{'EPSG':'32632','MINX':reproject('32632','4326',300000,4890240)[0],'MINY':reproject('32632','4326',300000,4890240)[1],'MAXX':reproject('32632','4326',409800,5000040)[0],'MAXY':reproject('32632','4326',409800,5000040)[1]
                },
        '32TLR':{'EPSG':'32632','MINX':reproject('32632','4326',300000,4990200)[0],'MINY':reproject('32632','4326',300000,4990200)[1],'MAXX':reproject('32632','4326',409800,5100000)[0],'MAXY':reproject('32632','4326',409800,5100000)[1]
                },
        '32TLS':{'EPSG':'32632','MINX':reproject('32632','4326',300000,5090220)[0],'MINY':reproject('32632','4326',300000,5090220)[1],'MAXX':reproject('32632','4326',409800,5200020)[0],'MAXY':reproject('32632','4326',409800,5200020)[1]
                }
    }
}



LANDSAT_wrs={
    "ALP":
    {
        "195029":["31TGJ",'31TGK','31TGL',"32TLP",'32TLQ','32TLR'],
        "195028":['31TGL','31TGM','32TLQ','32TLR','32TLS'],
        "196029":["31TGJ",'31TGK','31TGL','32TLQ','32TLR'],
        "196028":['31TGK','31TGL','31TGM','32TLQ','32TLR','32TLS'],
        "194029":["32TLP",'32TLQ','32TLR']
    },
    "PYR":
    {
        "200030":["30TXN",'30TYN'],
        "199030":["30TXN",'30TYN','31TCH'],
        "198031":['31TCH','31TDH'],
        "198030":['31TCH','31TDH'],
        "197031":['31TDH']
    }
}

LANDSAT_tiles={
    "ALP":
    {
        "31TGJ":["195029","196029"],
        '31TGK':["195029","196029","196028"],
        '31TGL':["195029","195028","196029","196028"],
        '31TGM':["195028","196028"],
        "32TLP":["194029","195029"],
        '32TLQ':["194029","195029","195028","196029","196028"],
        '32TLR':["194029","195029","195028","196029","196028"],
        '32TLS':["195028","196028"]
    },
    "PYR":
    {
        "30TXN":["200030","199030"],
        '30TYN':["200030","199030"],
        '31TCH':["199030","198031","198030"],
        '31TDH':["198031","198030","197031"]
    }
}

SPOT_tile={
    "ALP":
    {"KMIN":46,
     "KMAX":55,
     "JMIN":254,
     "JMAX":263
    },
    "PYR":
    {"KMIN":35,
     "KMAX":48,
     "JMIN":262,
     "JMAX":265
    }
}

SAFRAN_massifs={
    "ALP":
    {"Chablais",'Aravis','Mont-Blanc',"Haute-Tarentaise","Haute-Maurienne",
     "Beaufortain", "Vanoise","Maurienne","Thabor","Chartreuse",
     "Bauges","Oisans","Grandes-Rousses","Belledonne","Vercors",
     "Devoluy","Champsaur","Pelvoux","Queyras","Embrunnais Parpaillon",
     "Ubaye","Haut-Var Haut-Verdon","Mercantour"
    },
    "PYR":
    {"Pays-Basque","Aspe Ossau","Navarra","Jacetiana","Gallego",
     "Haute-Bigorre","Aure Louron","Luchonnais","Sobrarbe","Esera",
     "Couserans","Aran","Ribagorcana","Pallaresa","Haute-Ariege",
     "Andorre","Perafita","Orlu St-Barthelemy","Capcir Puymorens","Cadi Moixero",
     'Ter-Freser','Cerdagne Canigou'
    }
}

SAFRAN_tiles={
    "ALP":
    {"31TGM":["Chablais",'Aravis','Mont-Blanc'],
     "32TLS":["Chablais",'Aravis','Mont-Blanc'],
     "32TLR":["Chablais","Mont-Blanc","Haute-Tarentaise",
              "Haute-Maurienne","Beaufortain","Aravis",
              "Vanoise","Maurienne","Thabor"],
     "31TGL":["Mont-Blanc","Chablais","Aravis","Beaufortain",
              "Chartreuse","Haute-Maurienne","Vanoise",
              "Maurienne","Haute-Tarentaise","Bauges","Oisans",
              "Grandes-Rousses","Belledonne","Thabor","Vercors"],
     "31TGK":["Vercors","Oisans","Belledonne","Grandes-Rousses",
              "Devoluy","Thabor","Champsaur","Pelvoux","Queyras",
              "Embrunnais Parpaillon","Ubaye","Haut-Var Haut-Verdon",
              "Mercantour"],
     "32TLQ":["Queyras","Thabor","Pelvoux","Embrunnais Parpaillon",
              "Ubaye","Haut-Var Haut-Verdon","Mercantour"],
     "32TLP":["Haut-Var Haut-Verdon","Mercantour"],
     "31TGJ":["Haut-Var Haut-Verdon"]
     
    },
    "PYR":
    {"30TXN":["Pays-Basque","Aspe Ossau","Navarra","Jacetiana"],
     "30TYN":["Aspe Ossau","Jacetiana","Haute-Bigorre","Aure Louron",
              "Luchonnais","Gallego","Sobrarbe","Esera",
             "Couserans","Aran","Ribagorcana"],
     "31TCH":["Luchonnais","Esera","Couserans","Haute-Ariege","Andorre",
              "Aran","Ribagorcana","Pallaresa","Perafita","Orlu St-Barthelemy",
              "Capcir Puymorens","Cadi Moixero"],
     "31TDH":["Orlu St-Barthelemy","Capcir Puymorens","Cerdagne Canigou",
              "Ter-Freser","Cadi Moixero","Perafita","Haute-Ariege"]
    }
}

SAFRAN_wrs={
    "ALP":
    {
        "195029":["Haut-Var Haut-Verdon",'Mercantour',
                  'Ubaye',"Embrunnais Parpaillon",'Queyras',
                  'Pelvoux','Champsaur','Oisans','Devoluy','Oisans',
                  'Grandes-Rousses','Belledonne','Maurienne','Thabor',
                  'Haute-Maurienne','Vanoise','Chartreuse','Bauges','Beaufortain','Haute-Tarentaise'],
        "196029":["Haut-Var Haut-Verdon",'Mercantour',
                  'Ubaye',"Embrunnais Parpaillon",'Queyras',
                  'Pelvoux','Champsaur','Oisans','Devoluy','Oisans',
                  'Grandes-Rousses','Belledonne','Maurienne','Thabor',
                  'Haute-Maurienne','Vanoise','Chartreuse','Vercors'],
        "195028":['Chablais','Mont-Blanc','Aravis','Beaufortain',
                  'Haute-Tarentaise','Bauges','Vanoise','Haute-Maurienne',
                  'Maurienne','Belledonne','Chartreuse'],
        "196028":['Chablais','Mont-Blanc','Aravis','Beaufortain',
                  'Haute-Tarentaise','Bauges','Vanoise','Haute-Maurienne',
                  'Maurienne','Belledonne','Chartreuse','Vercors','Grandes-Rousses','Thabor'],
        "194029":["Haute-Maurienne",'Queyras','Mercantour','Ubaye','Haut-Var Haut-Verdon']
    },
    "PYR":
    {
        "200030":["Pays-Basque","Aspe Ossau","Navarra","Jacetiana","Gallego","Haute-Bigorre"],
        "199030":["Pays-Basque","Aspe Ossau","Navarra","Jacetiana","Gallego","Haute-Bigorre","Aure Louron",
                  "Luchonnais","Sobrarbe","Esera","Couserans","Aran","Ribagorcana","Pallaresa"],
        "198031":["Haute-Ariege","Andorre","Aran","Ribagorcana","Pallaresa","Perafita","Orlu St-Barthelemy",
              "Capcir Puymorens","Cadi Moixero",'Ter-Freser','Cerdagne Canigou'],
        "198030":["Haute-Ariege","Andorre","Aran","Ribagorcana","Pallaresa","Perafita","Orlu St-Barthelemy",
              "Capcir Puymorens","Cadi Moixero",'Ter-Freser','Cerdagne Canigou','Couserans','Orlu St-Barthelemy'],
        "197031":["Orlu St-Barthelemy","Capcir Puymorens","Cerdagne Canigou",
              "Ter-Freser","Cadi Moixero","Perafita"]
    }
}

In [ ]:
#extract swh snow maps from thabor

SWH_path = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/SWH_INFERENCE/TCD-BLUE_AVG-1200"
out_dir = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/TOP/THABOR/SNOW/SWH"
shp_path = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/TOP/THABOR/SHP/THABOR.mask.shp"
tmp_dir = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/TOP/TMP"
shapefile = gpd.read_file(shp_path)
lambert_epsg = "2154"
safran_lambert = shapefile.to_crs({'init': f"epsg:{lambert_epsg}"})

   
skip=0


for tile in ['31TGL']:
    list_swh = glob.glob(os.path.join(SWH_path,f'ALP/*/*/*/*/*/{tile}/*/*FSCTOC.tif'))
    already_done = glob.glob(os.path.join(out_dir,f'*{tile}*'))
    nb_swh = len(list_swh)
    nodata=0
    for i,swh in enumerate(list_swh):
        if out_dir+f"/THABOR_"+os.path.basename(swh) in already_done : 
            skip+=1
            print(f"{tile} {int((i+1)/nb_swh*100)}% skipped={skip}",end="                                                                          \r")
            continue
        print(f"{tile} {int((i+1)/nb_swh*100)}% skipped={skip}",end="                                                                          \r")
        

        tmp_path = tmp_dir+f"/swh_{i}_tmp.tif"
        lambert_geoms = [mapping((safran_lambert.geometry.values)[0])]
        bounds = safran_lambert.geometry.total_bounds

        tmp = gdal.Warp(tmp_path,swh,format="GTiff",
                        outputBounds= bounds,xRes=20,yRes=20,
                        dstSRS="EPSG:"+lambert_epsg,resampleAlg="near",
                        srcNodata=255,dstNodata=255)
        tmp_array = tmp.GetRasterBand(1).ReadAsArray()
        if np.all(tmp_array == 255): 
            os.system("rm "+tmp_path)
            continue
        else:
            with rasterio.open(tmp_path,'r') as src:
                out_image, out_transform = mask(src, lambert_geoms,nodata=255)
                out_meta = src.meta
                out_meta.update({"driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform})
            if np.all(out_image[0] == 255):
                os.system("rm "+tmp_path)
                continue
            else:
                mkdir_p(out_dir)
                out=out_dir+f"/THABOR_"+os.path.basename(swh)
                with rasterio.open(out, "w", **out_meta,compress='deflate') as dest:
                    dest.write(out_image)
                os.system("rm "+tmp_path)
                
                    
                    
                    
                    
    



In [ ]:
#extract dlr snow maps from thabor
import geopandas as gpd
from shapely.geometry import mapping, shape
from rasterio.mask import mask
import numpy as np
import os
import glob
from osgeo import osr, ogr, gdal
from osgeo.gdalnumeric import *
from osgeo.gdalconst import *

DLR_path = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/LANDSAT_QA_DLR"
out_dir = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/MISC/STCHRISTOPHE/DLR"
shp_path = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/MISC/VeneonStChristophe.kml"
tmp_dir = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/MISC/STCHRISTOPHE/TMP"
shapefile = gpd.read_file(shp_path,driver='LIBKML')
lambert_epsg = "4326"
safran_lambert = shapefile.to_crs({'init': f"epsg:{lambert_epsg}"})
lambert_geoms = [mapping((safran_lambert.geometry.values)[0])]
   


for mtn in ['ALP']:
    for wrs in ["196029","195029"]:
        list_dlr = glob.glob(os.path.join(DLR_path,f'{mtn}_LIS/*{wrs}/*FSCTOC.tif'))
        print(list_dlr)
        nb_dlr = len(list_dlr)
        nodata=0
        for i,dlr in enumerate(list_dlr):
            print(dlr)
            print(f"{wrs} {int((i+1)/nb_dlr*100)}% ",end="                                                                          \r")

            tmp_path = tmp_dir+f"/dlr_{i}_tmp.tif"
            
            bounds = safran_lambert.geometry.total_bounds
            print(bounds)
            tmp = gdal.Warp(tmp_path,dlr,format="GTiff",xRes=20,yRes=20,
                            dstSRS="EPSG:"+lambert_epsg,resampleAlg="near",
                            srcNodata=255,dstNodata=255)
            print(tmp)
            tmp_array = tmp.GetRasterBand(1).ReadAsArray()
            if np.all(tmp_array == 255): 
                os.system("rm "+tmp_path)
                continue
            else:
                with rasterio.open(tmp_path,'r') as src:
                    out_image, out_transform = mask(src, lambert_geoms,nodata=255)
                    out_meta = src.meta
                    out_meta.update({"driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform})
                if np.all(out_image[0] == 255):
                    os.system("rm "+tmp_path)
                    continue
                else:
                    mkdir_p(out_dir)
                    out=out_dir+f"/STCHRISTOPHE_"+os.path.basename(dlr)
                    with rasterio.open(out, "w", **out_meta,compress='deflate') as dest:
                        dest.write(out_image)

                    
                    
                    
                    
    



In [1]:
#extract swh snow maps from safran shapefile

SWH_path = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/SWH_INFERENCE/TCD-BLUE_AVG-1200"
out_dir = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/TOP/SAFRAN"
shp_path = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/SAFRAN/massifs_WGS84.shp"
tmp_dir = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/TOP/TMP"
shapefile = gpd.read_file(shp_path)
lambert_epsg = "2154"
safran_lambert = shapefile.to_crs({'init': f"epsg:{lambert_epsg}"})

   


for mtn in SAFRAN_tiles:
    for tile in SAFRAN_tiles[mtn]:
        if tile not in ["31TDH"]: continue
        skip=0
        data=0
        list_swh = glob.glob(os.path.join(SWH_path,f'{mtn}/*/*/*/*/*/{tile}/*/*FSCTOC.tif'))
        nb_swh = len(list_swh)
        nodata=0
        for i,swh in enumerate(list_swh):
            print(f"{tile} {int((i+1)/nb_swh*100)}% all = {data} skipped={skip} nodata={nodata}",end="                              \r")
            


            for j,safran in enumerate(SAFRAN_tiles[mtn][tile]):
                data+=1
                tmp_path = tmp_dir+f"/swh_{i}_{j}_tmp.tif"
                name = safran_lambert.loc[safran_lambert['title'] == safran, 'title_s'].values[0]
                out_path = out_dir+f"/{mtn}/{name}/SWH"
                already_done = glob.glob(os.path.join(out_path,f'{name}*{tile}*'))
                if out_path+f"/{name}_"+os.path.basename(swh) in already_done : 
                    skip+=1
                    print(f"{tile} {int((i+1)/nb_swh*100)}% all = {data} skipped={skip} nodata={nodata}",end="                              \r")
                    continue
                lambert_geoms = [mapping(((safran_lambert.query(f"title == '{safran}'")).geometry.values)[0])]
                bounds = safran_lambert.loc[safran_lambert['title'] == safran, 'geometry'].total_bounds
                
                tmp = gdal.Warp(tmp_path,swh,format="GTiff",
                                outputBounds= bounds,xRes=20,yRes=20,
                                dstSRS="EPSG:"+lambert_epsg,resampleAlg="near",
                                srcNodata=255,dstNodata=255)
                tmp_array = tmp.GetRasterBand(1).ReadAsArray()
                if np.all(tmp_array == 255): 
                    os.system("rm "+tmp_path)
                    nodata+=1
                    print(f"{tile} {int((i+1)/nb_swh*100)}% all = {data} skipped={skip} nodata={nodata}",end="                              \r")
                    continue
                else:
                    with rasterio.open(tmp_path,'r') as src:
                        out_image, out_transform = mask(src, lambert_geoms,nodata=255)
                        out_meta = src.meta
                        out_meta.update({"driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        "transform": out_transform})
                    if np.all(out_image[0] == 255):
                        os.system("rm "+tmp_path)
                        nodata+=1
                        print(f"{tile} {int((i+1)/nb_swh*100)}% all = {data} skipped={skip} nodata={nodata}",end="                              \r")
                        continue
                    else:
                        mkdir_p(out_path)
                        out=out_path+f"/{name}_"+os.path.basename(swh)
                        with rasterio.open(out, "w", **out_meta,compress='deflate') as dest:
                            dest.write(out_image)
                        os.system("rm "+tmp_path)
                
                    
                    
                    
                    
    



NameError: name 'gpd' is not defined

In [ ]:
#extract dlr snow maps from safran shapefile

DLR_path = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/LANDSAT_QA_DLR"
out_dir = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/TOP/SAFRAN"
shp_path = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/SAFRAN/massifs_WGS84.shp"
tmp_dir = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/TOP/TMP"
shapefile = gpd.read_file(shp_path)
lambert_epsg = "2154"
safran_lambert = shapefile.to_crs({'init': f"epsg:{lambert_epsg}"})

   


for mtn in ["PYR"]:
    for wrs in ["198030","197031"]:
        list_dlr = glob.glob(os.path.join(DLR_path,f'{mtn}_LIS/*{wrs}/*FSCTOC.tif'))
        nb_dlr = len(list_dlr)
        data=0
        nodata=0
        for i,dlr in enumerate(list_dlr):
            print(f"{wrs} {int((i+1)/nb_dlr*100)}% all = {data} nodata={nodata}",end="                                        \r")
            for j,safran in enumerate(SAFRAN_wrs[mtn][wrs]):
                data+=1
                tmp_path = tmp_dir+f"/dlr_{i}_{j}_tmp.tif"
                name = safran_lambert.loc[safran_lambert['title'] == safran, 'title_s'].values[0]
                out_path = out_dir+f"/{mtn}/{name}/DLR"
                lambert_geoms = [mapping(((safran_lambert.query(f"title == '{safran}'")).geometry.values)[0])]
                bounds = safran_lambert.loc[safran_lambert['title'] == safran, 'geometry'].total_bounds
                
                tmp = gdal.Warp(tmp_path,dlr,format="GTiff",
                                outputBounds= bounds,xRes=20,yRes=20,
                                dstSRS="EPSG:"+lambert_epsg,resampleAlg="near",
                                srcNodata=255,dstNodata=255)
                tmp_array = tmp.GetRasterBand(1).ReadAsArray()
                if np.all(tmp_array == 255): 
                    os.system("rm "+tmp_path)
                    nodata+=1
                    print(f"{wrs} {int((i+1)/nb_dlr*100)}% all = {data} nodata={nodata}",end="                                        \r")
                    continue
                else:
                    with rasterio.open(tmp_path,'r') as src:
                        out_image, out_transform = mask(src, lambert_geoms,nodata=255)
                        out_meta = src.meta
                        out_meta.update({"driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        "transform": out_transform})
                    if np.all(out_image[0] == 255):
                        os.system("rm "+tmp_path)
                        nodata += 1
                        print(f"{wrs} {int((i+1)/nb_dlr*100)}% all = {data} nodata={nodata}",end="                                        \r")
                        continue
                    else:
                        mkdir_p(out_path)
                        out=out_path+f"/{name}_"+os.path.basename(dlr)
                        with rasterio.open(out, "w", **out_meta,compress='deflate') as dest:
                            dest.write(out_image)
                            print(f"{wrs} {int((i+1)/nb_dlr*100)}% all = {data} nodata={nodata}",end="                                        \r")
                
                    
                    
                    
                    
    



In [ ]:
#extract synthesis maps from safran shapefile

SWH_path = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/SYNTHESIS/LANDSAT_SWH/TCD-BLUE_AVG-1200"
out_dir = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/TOP/SYNTHESIS"
shp_path = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/SAFRAN/massifs_WGS84.shp"
tmp_dir = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/TOP/TMP"
shapefile = gpd.read_file(shp_path)
lambert_epsg = "2154"
safran_lambert = shapefile.to_crs({'init': f"epsg:{lambert_epsg}"})

   


for mtn in ["PYR","ALP"]:
    for tile in SAFRAN_tiles[mtn]:
        for metric in ["NOBS","NSP","SCD","SMOD","SOD"]:

            list_swh = glob.glob(os.path.join(SWH_path,f'{mtn}/{tile}/*/*{metric}*'))
            nb_swh = len(list_swh)

            for i,swh in enumerate(list_swh):
                print(f"{mtn} {tile} {metric} {int((i+1)/nb_swh*100)}%",end="                              \r")

                for j,safran in enumerate(SAFRAN_tiles[mtn][tile]):
                    tmp_path = tmp_dir+f"/swh_{i}_{j}_tmp.tif"
                    name = safran_lambert.loc[safran_lambert['title'] == safran, 'title_s'].values[0]
                    out_path = out_dir+f"/{mtn}/{name}/{metric}"

                    lambert_geoms = [mapping(((safran_lambert.query(f"title == '{safran}'")).geometry.values)[0])]
                    bounds = safran_lambert.loc[safran_lambert['title'] == safran, 'geometry'].total_bounds

                    tmp = gdal.Warp(tmp_path,swh,format="GTiff",
                                    outputBounds= bounds,xRes=20,yRes=20,
                                    dstSRS="EPSG:"+lambert_epsg,resampleAlg="near",
                                    srcNodata=400,dstNodata=400)
                    tmp_array = tmp.GetRasterBand(1).ReadAsArray()
                    if np.all(tmp_array == 400): 
                        os.system("rm "+tmp_path)
                        continue
                    else:
                        with rasterio.open(tmp_path,'r') as src:
                            out_image, out_transform = mask(src, lambert_geoms,nodata=400)
                            out_meta = src.meta
                            out_meta.update({"driver": "GTiff",
                            "height": out_image.shape[1],
                            "width": out_image.shape[2],
                            "transform": out_transform})
                        if np.all(out_image[0] == 400):
                            os.system("rm "+tmp_path)
                            continue
                        else:
                            mkdir_p(out_path)
                            out=out_path+f"/{name}_"+os.path.basename(swh)
                            with rasterio.open(out, "w", **out_meta,compress='deflate') as dest:
                                dest.write(out_image)
                            os.system("rm "+tmp_path)

                    
                    
                    
                    
    



In [7]:
#merge same date & same massif SWH rasters
SWH_path = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/TOP/SAFRAN"
out_dir = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/TOP/MERGED"

for mtn in os.listdir(SWH_path):
    for massif in os.listdir(os.path.join(SWH_path,mtn)):
        if massif != "AuLou" : continue
        out_path = os.path.join(out_dir,mtn,massif,"SWH")
        mkdir_p(out_path)
        #get list of products
        swh_list = glob.glob(os.path.join(SWH_path,mtn,massif,"SWH",'*'))
        #sort by dates
        dict_dates={}
        print(f"{mtn} {massif} sort dates",end="                                        \r")
        for swh in swh_list:
            date_swh = getDateFromStr(swh)
            if date_swh not in dict_dates : dict_dates[date_swh] = []
            dict_dates[date_swh].append(swh)
        # merge by dates
        len_dates = len(dict_dates)
        for i,date_swh in enumerate(dict_dates):
            to_merge = " ".join(dict_dates[date_swh])
            out = os.path.join(out_path,f"{massif}_SWH_{date_swh.strftime('%Y%m%d')}_SNOW.tif")
            os.system(f"gdal_merge.py -n 255 -a_nodata 255 -quiet -co COMPRESS=DEFLATE -o {out} {to_merge}")
            print(f"{mtn} {massif} merge dates {int((i+1)/len_dates*100)}%",end="                                        \r")

In [6]:
#merge same date & same massif SWH thabor
SWH_path = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/TOP/THABOR/SNOW"
out_dir = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/TOP/THABOR_MERGED"

for sat in os.listdir(SWH_path):
    out_path = os.path.join(out_dir,sat)
    mkdir_p(out_path)
    #get list of products
    swh_list = glob.glob(os.path.join(SWH_path,sat,'*'))
    #sort by dates
    dict_dates={}
    print(f"{sat} sort dates",end="                                        \r")
    for swh in swh_list:
        date_swh = getDateFromStr(swh)
        if date_swh not in dict_dates : dict_dates[date_swh] = []
        dict_dates[date_swh].append(swh)
    # merge by dates
    len_dates = len(dict_dates)
    for i,date_swh in enumerate(dict_dates):
        to_merge = " ".join(dict_dates[date_swh])
        out = os.path.join(out_path,f"THABOR_{sat}_{date_swh.strftime('%Y%m%d')}_SNOW.tif")
        os.system(f"gdal_merge.py -n 255 -a_nodata 255 -quiet -co COMPRESS=DEFLATE -o {out} {to_merge}")
        print(f"{sat} merge dates {int((i+1)/len_dates*100)}%",end="                                        \r")

In [ ]:
#merge same date & same massif synthesis rasters
SWH_path = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/TOP/SYNTHESIS"
out_dir = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/TOP/SYNTHESIS_MERGED"

for mtn in os.listdir(SWH_path):
    for massif in os.listdir(os.path.join(SWH_path,mtn)):
        for metric in os.listdir(os.path.join(SWH_path,mtn,massif)):
            out_path = os.path.join(out_dir,mtn,massif,metric)
            mkdir_p(out_path)
            #get list of products
            swh_list = glob.glob(os.path.join(SWH_path,mtn,massif,metric,'*'))
            #sort by dates
            dict_dates={}
            print(f"{mtn} {massif} sort dates",end="                                        \r")
            for swh in swh_list:
                date_swh = getDateFromStr(swh)
                if date_swh.year not in dict_dates : dict_dates[date_swh.year] = []
                dict_dates[date_swh.year].append(swh)
            # merge by dates
            len_dates = len(dict_dates)
            for i,date_swh in enumerate(dict_dates):
                to_merge = " ".join(dict_dates[date_swh])
                out = os.path.join(out_path,f"{massif}_SWH_{date_swh.strftime('%Y%m%d')}_SNOW.tif")
                os.system(f"gdal_merge.py -n 400 -a_nodata 400 -quiet -co COMPRESS=DEFLATE -o {out} {to_merge}")
                print(f"{mtn} {massif} merge dates {int((i+1)/len_dates*100)}%",end="                                        \r")

In [11]:
os.listdir(os.path.join("/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/TOP/SAFRAN","PYR"))

['AspOs',
 'Couse',
 'Navar',
 'Lucho',
 'OrSBa',
 'Jacet',
 'Ribag',
 'P-Bas',
 'Aran',
 'Palla',
 'Andor',
 'Galle',
 'CerCa',
 'Peraf',
 'Sobra',
 'CaPuy',
 'H-Big',
 'TerFr',
 'Esera',
 'H-Ari',
 'CaMoi',
 'AuLou']

In [ ]:
#merge same date & same massif DLR rasters
SWH_path = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/TOP/SAFRAN"
out_dir = "/home/ad/barrouz/zacharie/TIMESERIES_PROJECT/TOP/MERGED"

for mtn in os.listdir(SWH_path):
    for massif in os.listdir(os.path.join(SWH_path,mtn)):
        out_path = os.path.join(out_dir,mtn,massif,"DLR")
        mkdir_p(out_path)
        #get list of products
        swh_list = glob.glob(os.path.join(SWH_path,mtn,massif,"DLR",'*'))
        #sort by dates
        dict_dates={}
        print(f"{mtn} {massif} sort dates",end="                                        \r")
        for swh in swh_list:
            date_swh = getDateFromStr(swh)
            if date_swh not in dict_dates : dict_dates[date_swh] = []
            dict_dates[date_swh].append(swh)
        # merge by dates
        len_dates = len(dict_dates)
        for i,date_swh in enumerate(dict_dates):
            to_merge = " ".join(dict_dates[date_swh])
            out = os.path.join(out_path,f"{massif}_DLR_{date_swh.strftime('%Y%m%d')}_SNOW.tif")
            os.system(f"gdal_merge.py -n 255 -a_nodata 255 -quiet -co COMPRESS=DEFLATE -o {out} {to_merge}")
            print(f"{mtn} {massif} merge dates {int((i+1)/len_dates*100)}%",end="                                        \r")